Author

Melika Maleki Rad - Final Project 2024
SBU University
Aerospace Engineering

# Predicting the mechanical behavior of composite materials using machine learning
Tensorflow
https://www.tensorflow.org/tutorials/generative/pix2pix

## Import libraries

In [55]:
import tensorflow as tf
import os
import pathlib
import time
import datetime
from pathlib import Path
#from google.colab import drive
from tensorflow.keras.models import Model as model
from matplotlib import pyplot as plt
from IPython import display
#drive.mount('/content/drive')

## Load the dataset

The Dataset is generated by ANSYS

In [56]:
#dataset_name = "facades" #@param ["cityscapes", "edges2handbags", "edges2shoes", "facades", "maps", "night2day"]


In [57]:
# _URL = f'http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/{dataset_name}.tar.gz'

# path_to_zip = tf.keras.utils.get_file(
#     fname=f"{dataset_name}.tar.gz",
#     origin=_URL,
#     extract=True)

# path_to_zip  = pathlib.Path(path_to_zip)

# PATH = path_to_zip.parent/dataset_name
PATH = Path('YOUR PATH')
list(PATH.parent.iterdir())

In [59]:
# sample_image = tf.io.read_file(str(PATH / 'train/1.jpg'))
# sample_image = tf.io.decode_jpeg(sample_image)
# print(sample_image.shape)
# plt.figure()
# plt.imshow(sample_image)

In [61]:
def load(image_file):
  image = tf.io.read_file(image_file)
  image = tf.image.decode_png(image)

  w = tf.shape(image)[1]

  w = w // 2
  real_image = image[:, w:, :]
  real_image = tf.cast(real_image, tf.float32)

  return input_image, real_image
  

In [62]:
# inp, re = load(str(PATH / 'train/100.jpg'))
# Casting to int for matplotlib to display the images
# plt.figure()
# plt.imshow(inp / 255.0)
# plt.figure()
# plt.imshow(re / 255.0)

In [63]:
BUFFER_SIZE = 400
BATCH_SIZE = 1
IMG_WIDTH = 512
IMG_HEIGHT = 512

In [64]:
  
def resize(input_image, real_image, height, width):
  input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  real_image = tf.image.resize(real_image, [height, width],
                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  return input_image, real_image
  

In [65]:
def random_crop(input_image, real_image):
  stacked_image = tf.stack([input_image, real_image], axis=0)
      stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image[0], cropped_image[1]

In [66]:
def normalize(input_image, real_image):
  input_image = (input_image / 127.5) - 1
  real_image = (real_image / 127.5) - 1

  return input_image, real_image

In [67]:
@tf.function()
def random_jitter(input_image, real_image):
  # resizing to 286 x 286 x 3
  input_image, real_image = resize(input_image, real_image, 1024, 1024)

  # randomly cropping to 256 x 256 x 3
  input_image, real_image = random_crop(input_image, real_image)

  if tf.random.uniform(()) > 0.5:
    # random mirroring
    input_image = tf.image.flip_left_right(input_image)
    real_image = tf.image.flip_left_right(real_image)

  return input_image, real_image

In [68]:
# plt.figure(figsize=(6, 6))
# for i in range(4):
#   rj_inp, rj_re = random_jitter(inp, re)
#   plt.subplot(2, 2, i + 1)
#   plt.imshow(rj_inp / 255.0)
#   plt.axis('off')
# plt.show()


In [69]:
 
def load_image_train(image_file):
  input_image, real_image = load(image_file)
  input_image, real_image = random_jitter(input_image, real_image)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image


In [70]:

def load_image_test(image_file):
  input_image, real_image = load(image_file)
  input_image, real_image = resize(input_image, real_image,
                                   IMG_HEIGHT, IMG_WIDTH)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image
  

In [71]:
train_dataset = tf.data.Dataset.list_files(str('YOUR PATH/*.png'))
train_dataset = train_dataset.map(load_image_train,
                                  num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
test_dataset = tf.data.Dataset.list_files('YOUR PATH/*.png', shuffle=False)
test_dataset = test_dataset.map(load_image_test)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [72]:
num_data = 40

In [73]:
OUTPUT_CHANNELS = 3

In [74]:
def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [75]:
# down_model = downsample(3, 4)
# down_result = down_model(tf.expand_dims(inp, 0))
# print (down_result.shape)

In [76]:
 
def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [77]:
# up_model = upsample(3, 4)
# up_result = up_model(down_result)
# print (up_result.shape)

In [78]:

def Generator():
  inputs = tf.keras.layers.Input(shape=[512,512,3])

  down_stack = [
    downsample(32, 4, apply_batchnorm=False), #(bs, 256, 256, 32)
    downsample(64, 4), # (bs, 128, 128, 64)
    downsample(128, 4), # (bs, 64, 64, 128)
    downsample(256, 4), # (bs, 32, 32, 256)
    downsample(512, 4), # (bs, 1, 1, 512)
  ]

  up_stack = [
    upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
    upsample(512, 4), # (bs, 16, 16, 1024
    upsample(32, 4), # (bs, 256, 256, 64)
  ] 

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh') # (bs, 512, 512, 3)

  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)


In [ ]:
generator = Generator()
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

In [80]:
# gen_output = generator(inp[tf.newaxis, ...], training=False)
# plt.imshow(gen_output[0, ...])

In [81]:
LAMBDA = 100

In [82]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)


In [83]:

def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss, gan_loss, l1_loss

In [84]:
 
def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[512, 512, 3], name='input_image')
  tar = tf.keras.layers.Input(shape=[512, 512, 3], name='target_image')

  x = tf.keras.layers.concatenate([inp, tar]) # (bs, 512, 512, channels*2)

  down1 = downsample(32, 4, False)(x) #(bs, 256, 256, 32)
  down2 = downsample(64, 4)(down1) # (bs, 128, 128, 64)
  down3 = downsample(128, 4)(down2) # (bs, 64, 64, 128)
  down4 = downsample(256, 4)(down3) # (bs, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down4) # (bs, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)

  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

  return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [85]:
discriminator = Discriminator()
#tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

In [86]:
# disc_out = discriminator([inp[tf.newaxis, ...], gen_output], training=False)
# plt.imshow(disc_out[0, ..., -1], vmin=-20, vmax=20, cmap='RdBu_r')
# plt.colorbar()

In [87]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss


In [88]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)


In [89]:
checkpoint_dir = 'YOUR PATH'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)


In [ ]:
count = 0
for inp, tar in test_dataset.take(num_data):
  print(count)
  count += 1


def generate_images(model, test_input, tar, count):
  prediction = model(test_input, training=True)
  plt.figure(figsize=(25, 25))
  #plt.figure(figsize=(6.5, 6.5))

  plt.imshow(tar[0] * 0.5 + 0.5)
  plt.axis('off')
  plt.savefig('YOUR PATH' + str(count) + '.png', bbox_inches='tight',pad_inches=-0.1)
  display_list = [test_input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']


  for i in range(3):
    plt.subplot(1, 3, i+1)
    #plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.savefig('YOUR PATH' + str(count) + '.png')


In [93]:
# def predict_images(model, test_input):
#   prediction = model(test_input, training=True)
#   display_list = [test_input[0], prediction[0]]
#   to_image = Image.new('RGB', (IMG_WIDTH * 2, IMG_WIDTH))
#   for i in range(2):
#     # getting the pixel values between [0, 1] to plot it.
#     img_temp = tf.keras.preprocessing.image.array_to_img(display_list[i] * 0.5 + 0.5)
#     to_image.paste(img_temp, (i * IMG_WIDTH, 0))
#   return to_image

@tf.function()
def random_jitter(input_image, real_image):
  # Resizing to 286x286
  input_image, real_image = resize(input_image, real_image, 286, 286)

  # Random cropping back to 256x256
  input_image, real_image = random_crop(input_image, real_image)

  if tf.random.uniform(()) > 0.5:
    # Random mirroring
    input_image = tf.image.flip_left_right(input_image)
    real_image = tf.image.flip_left_right(real_image)

  return input_image, real_image

In [94]:
    
#epochs=300
  
@tf.function
def train_step(input_image, target, epoch,  generator, discriminator, generator_optimizer, discriminator_optimizer, summary_writer):
  
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))
  with summary_writer.as_default():
    tf.summary.scalar('gen_total_loss', gen_total_loss, step=epoch)
    tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
    tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=epoch)
    tf.summary.scalar('disc_loss', disc_loss, step=epoch)


In [95]:

def fit(train_ds, epochs, test_ds, generator, discriminator, generator_optimizer, discriminator_optimizer, checkpoint, summary_writer):
  for epoch in range(epochs):
    start = time.time()

    display.clear_output(wait=True)

    #for example_input, example_target in test_ds.take(1):
    #  generate_images(generator, example_input, example_target)
    print("Epoch: ", epoch)

    # Train
    for n, (input_image, target) in train_ds.enumerate():
      print('.', end='')
      if (n+1) % 100 == 0:
        print()
      train_step(input_image, target, epoch, generator, discriminator, generator_optimizer, discriminator_optimizer, summary_writer)
    print()

    # saving (checkpoint) the model every 20 epochs
    if (epoch + 1) % 20 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                        time.time()-start))
  checkpoint.save(file_prefix = checkpoint_prefix)


In [96]:
# %load_ext tensorboard
# %tensorboard --logdir {log_dir}

In [ ]:
log_dir="logs/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
   #fit(train_ds=train_dataset, epochs=50, test_ds=test_dataset, generator=generator, discriminator=discriminator,
    generator_optimizer=generator_optimizer, discriminator_optimizer=discriminator_optimizer,
    checkpoint=checkpoint, summary_writer=summary_writer  )
 fit(train_ds=train_dataset, test_ds=test_dataset, steps = 10000)

In [ ]:
!ls {checkpoint_dir}

In [ ]:
# Restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
# Run the trained model on a few examples from the test set
# for inp, tar in prediction_dataset.take(1):
#   predict_images(generator, inp, tar,count)

BATCH_SIZEII=1
prediction_dataset =  tf.data.Dataset.list_files('YOUR PATH', shuffle=False)
prediction_dataset = prediction_dataset.map(load_image_test)
prediction_dataset = prediction_dataset.batch(BATCH_SIZE)

import PIL.Image as Image
def predict_images(model, test_input ):
  prediction = model(test_input, training=True)
  #display_list = [test_input[0], prediction[0]]
  to_image = Image.new('RGB', (512 * 2, 512))
  for i in range(2):
    # getting the pixel values between [0, 1] to plot it.
    img_temp = tf.keras.preprocessing.image.array_to_img(display_list[i] * 0.5 + 0.5)
    to_image.paste(img_temp, (i * 512, 0))
  return to_image


 
for inp, tar in prediction_dataset.take(1):
  
  img = predict_images(generator, inp) ### return PIL image object
  
  img.save('YOUR PATH/predicted.png') ### save PIL image object
  

In [ ]:
# import PIL.Image as Image
# import tensorflow as tf
# import numpy as np

# IMG_WIDTH = 256  # Define IMG_WIDTH if not already defined

# def predict_images(model, test_input):
#     # Convert input to RGB if it has an alpha channel
#     if test_input.shape[-1] == 4:
#         test_input = tf.image.rgb_to_grayscale(test_input)
    
#     prediction = model(test_input, training=True)
#     display_list = [test_input[0], prediction[0]]
#     to_image = Image.new('RGB', (IMG_WIDTH * 2, IMG_WIDTH))
    
#     for i in range(2):
#         # Getting the pixel values between [0, 1] to plot it.
#         img_temp = tf.keras.preprocessing.image.array_to_img(display_list[i] * 0.5 + 0.5)
#         to_image.paste(img_temp, (i * IMG_WIDTH, 0))
    
#     return to_image

# # Your existing code
# for inp, tar in prediction_dataset.take(1):
#     img = predict_images(generator, inp)  # Ensure this returns a PIL Image object
    
#     # Save the image
#     img.save('YOUR PATH')
